# Atomic and molecular integral with pySCF

In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

In [2]:
import qib

### Construct the molecule

In [3]:
atom = '''H 0 0 0; H 0 0 0.735'''
basis = 'sto-6g'
unit = 'angstrom'
charge = 0
spin = 0
verbose = 0

mol = gto.Mole()
mol.build(atom    = atom,
          basis   = basis,
          charge  = charge,
          spin    = spin,
          unit    = unit,
          verbose = verbose)

### Create the atomic integrals

In [9]:
# costant energy term (nuclear repulsion and kinetic term)
h0 = mol.get_enuc()
# one body integral in the spin-orbital basis (electronic kinetic term and electronic-nuclear attraction)
h1 = np.kron(mol.get_hcore(), np.identity(2))
# two body integral in the spin-orbital basis (electronic repulsion)
h2 = mol.intor('int2e_spinor')

# create a MolecularHamiltonian object starting from the atomic integrals
latt = qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2)
field = qib.field.Field(qib.field.ParticleType.FERMION, latt)
H = qib.operator.MolecularHamiltonian(field, h0, h1, h2.transpose(0, 2, 1, 3))

In [10]:
print(H.as_matrix())

  (0, 0)	(0.7199689944489797+0j)
  (1, 1)	(-0.4084740771933558+0j)
  (1, 4)	(-0.9679805651651898+0j)
  (2, 2)	(-0.4084740771933558+0j)
  (2, 8)	(-0.9679805651651898+0j)
  (3, 3)	(-0.7619186275380483+0j)
  (3, 6)	(0.5207162000150318+0j)
  (3, 9)	(-0.5207162000150318+0j)
  (3, 12)	(0.30060831485471257+0j)
  (4, 1)	(-0.9679805651651898+0j)
  (4, 4)	(-0.4084740771933558+0j)
  (5, 5)	(-1.2656469256394343+0j)
  (6, 3)	(0.520716200015032+0j)
  (6, 6)	(-0.9650386107847218+0j)
  (6, 9)	(-0.30060831485471257+0j)
  (6, 12)	(0.520716200015032+0j)
  (7, 7)	(-1.0472129379331578+0j)
  (7, 13)	(0.07345183486487394+0j)
  (8, 2)	(-0.9679805651651898+0j)
  (8, 8)	(-0.4084740771933558+0j)
  (9, 3)	(-0.520716200015032+0j)
  (9, 6)	(-0.30060831485471257+0j)
  (9, 9)	(-0.9650386107847218+0j)
  (9, 12)	(-0.520716200015032+0j)
  (10, 10)	(-1.2656469256394343+0j)
  (11, 11)	(-1.047212937933158+0j)
  (11, 14)	(0.073451834864874+0j)
  (12, 3)	(0.30060831485471257+0j)
  (12, 6)	(0.5207162000150318+0j)
  (12, 9)	(-

### Hartree-Fock calculation

In [11]:
# RHF == restricted HF (look also ROHF, UHF...)
mf = scf.RHF(mol)
# performs the HF calculation
mf.kernel()
# gets the coefficient matrix for the molecular orbitals
coeff = mf.mo_coeff
print(coeff)

[[ 0.5483259   1.21806548]
 [ 0.5483259  -1.21806548]]


### Molecular integrals

In [12]:
spin_coeff = np.kron(coeff, np.identity(2))
h1_mo = np.einsum('ji,jk,kl->il', spin_coeff.conj(), h1, spin_coeff)
h2_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', h2, spin_coeff, spin_coeff, spin_coeff, spin_coeff)
H.tkin = h1_mo
H.vint = h2_mo.transpose(0, 2, 3, 1)
print(h1_mo)
print(h2_mo)

[[-1.26062688e+00  0.00000000e+00 -3.33066907e-16  0.00000000e+00]
 [ 0.00000000e+00 -1.26062688e+00  0.00000000e+00 -3.33066907e-16]
 [-1.11022302e-16  0.00000000e+00 -4.76151148e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.11022302e-16  0.00000000e+00 -4.76151148e-01]]
[[[[ 6.75656092e-01+0.j  0.00000000e+00+0.j -2.77555756e-17+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  6.75656092e-01+0.j  0.00000000e+00+0.j
    -2.77555756e-17+0.j]
   [ 2.77555756e-17+0.j  0.00000000e+00+0.j  6.65257654e-01+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  2.77555756e-17+0.j  0.00000000e+00+0.j
     6.65257654e-01+0.j]]

  [[ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]
   [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
     0.00000000e+00+0.j]]

  [[ 2.77555756

### Jordan-Wigner mapping

In [13]:
pauli_op = qib.transform.jordan_wigner_encode_field_operator(H.as_field_operator())

# We must assign a lattice to the pauli operator and the number of sites in the lattice must be exactly like the size of the Pauli strings
field_q = qib.field.Field(qib.field.ParticleType.QUBIT, qib.lattice.LayeredLattice(qib.lattice.FullyConnectedLattice((2,)), 2))
pauli_op.set_field(field_q)
print(pauli_op)

Pauli operator consisting of weighted Pauli strings
 (-0.09820181960059482+0j)*IIII
  (0.17407509959369882+0j)*ZIII
  (0.17407509959369882+0j)*IZII
 (-0.22429329823657265+0j)*IIZI
  (-0.2242932982365726+0j)*IIIZ
  (0.16891402311629494+0j)*ZZII
  (0.12100990274925519+0j)*ZIZI
  (0.16631441337615077+0j)*ZIIZ
 (0.045304510626895576+0j)*XYYX
(-0.045304510626895576+0j)*YYXX
(-0.045304510626895576+0j)*XXYY
 (0.045304510626895576+0j)*YXXY
  (0.16631441337615077+0j)*IZZI
  (0.12100990274925519+0j)*IZIZ
  (0.17504455596821475+0j)*IIZZ

